## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import os

from sklearn.model_selection import train_test_split

import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator

## Defining The Dataframe

In [2]:
#Managing the data path
train_path='../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train'

data = {'label':[],'file':[]}
for folder in os.listdir(train_path):
    label_folder = os.path.join(train_path, folder)
    for file in os.listdir(label_folder):
        data['label'].append(folder)
        data['file'].append(os.path.join(train_path,folder,file))

dftrain=pd.DataFrame(data)
dftrain.head()

In [3]:
plt.imshow(mpimg.imread(dftrain.iloc[0,1]))

## Preprocessing

In [4]:
dtrain, dtest = train_test_split(dftrain, test_size=0.2, random_state=5)

In [5]:
img_width, img_height=64,64

traindatagen = ImageDataGenerator(rescale=1./255)
testdatagen = ImageDataGenerator(rescale=1./255)

traingen = traindatagen.flow_from_dataframe(
                            dataframe=dtrain,
                            x_col='file',
                            y_col='label',
                            target_size=(img_width,img_height),
                            class_mode='categorical'
                            )

testgen = testdatagen.flow_from_dataframe(
                            dataframe=dtest,
                            x_col='file',
                            y_col='label',
                            target_size=(img_width,img_height),
                            class_mode='categorical'
                            )



## Defining The Model

In [6]:
model = Sequential()

model.add(Conv2D(filters=32,kernel_size=[5,5], activation='relu',input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(filters=64,kernel_size=[3,3], activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))

model.add(Conv2D(filters=64,kernel_size=[3,3], activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(29, activation = "softmax"))

In [7]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
#Early Stop Callback
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callback = EarlyStopping(monitor='val_loss', patience=2)

In [9]:
history = model.fit(traingen,
                   epochs=50,
                   validation_data=testgen,
                   callbacks=[callback])

In [10]:
#model.save('asl_model.h5')

##Evaluating The Model

In [2]:
score=pd.DataFrame(history.history)
score.head()

In [ ]:
sns.lineplot(data=score[['acc','val_acc']])